# EDA
The aim of this notebook is to figure out:

- How to collect raw tweets (with geolocation, handles, hashtags, mentions)
- Which people live in Argentina (maybe through user locations)
- How to track their movements over time
- How long to track their movements over time

In [ ]:
import pandas as pd # for dataframes
#import json
import snscrape.modules.twitter as sntwitter
from _datetime import datetime, timedelta
#from multiprocessing import Pool
#import tqdm # for progress bar
import pytz # for current time
import pandas as pd
#import itertools

In [ ]:
# set datetime parameters for scraping tweets

now = datetime.now().replace(tzinfo=pytz.utc)
start_date = (now - timedelta(days=1)).strftime('%Y-%m-%d')
end_date = (now + timedelta(days=1)).strftime('%Y-%m-%d')
date_window = str(start_date) + "-" + str(end_date)
scraper = sntwitter.TwitterSearchScraper

In [ ]:
# the data that currently is in Equinor/data/input_data/argentina_daily_tweets.csv is df
# this was done on 27.09.2022 for 1 day, and yielded 300k+ tweets

df = pd.DataFrame(scraper(f'near:Argentina within:10km since:{start_date} until:{end_date}').get_items())
print(df)

In [ ]:
# creating a new dataframe so we don't mess up the original
# putting in extra columns with user information

df_coord = df
df_coord['user_username'] =  df_coord['user'].apply(lambda x: x['username'])
df_coord['user_id'] =  df_coord['user'].apply(lambda x: x['id'])
df_coord['user_verified'] =  df_coord['user'].apply(lambda x: x['verified'])
df_coord['user_created'] =  df_coord['user'].apply(lambda x: x['created'])
df_coord['user_location'] =  df_coord['user'].apply(lambda x: x['location'])
df_coord['user_followers_count'] = df_coord['user'].apply(lambda x: x['followersCount'])
df_coord['user_friends_count'] = df_coord['user'].apply(lambda x: x['friendsCount'])
df_coord['user_statuses_count'] = df_coord['user'].apply(lambda x: x['statusesCount'])
df_coord['user_favourites_count'] =  df_coord['user'].apply(lambda x: x['favouritesCount'])
df_coord['user_listed_count'] =  df_coord['user'].apply(lambda x: x['listedCount'])
df_coord['user_media_count'] =  df_coord['user'].apply(lambda x: x['mediaCount'])
df_coord['user_protected'] = df_coord['user'].apply(lambda x: x['protected'])
df_coord['user_label'] =  df_coord['user'].apply(lambda x: x['label'])

# saved this to 
df_coord.to_csv('../../../Equinor/data/input_data/argentina_modified_daily_tweets.csv', index=False)

In [ ]:
# exploring the data found
df.user_username.value_counts(dropna=False) # not too many from each, which is good
df.user_location.value_counts(dropna=False) # mostly argentina, with a few more specific locations within argentina; needs filtered
df.user_verified.value_counts(dropna=False) # as expected, most are unverified

In [ ]:
# to be done next time for more precision
df2 = pd.DataFrame(scraper(f'geocode:-31.416668,-64.183334,10km since:{start_date} until:{end_date}').get_items())
print(df2)